# Data Science Challenge

**Objetivo: Desarrollar un script en Python que utilice aprendizaje automático para predecir si un cliente presentará un reclamo de seguro en el próximo año.**

Descripción: La compañía aseguradora está interesada en predecir si un cliente presentará un reclamo de seguro en el próximo año para poder tomar medidas preventivas y mejorar la satisfacción del cliente. La compañía ha recopilado datos históricos de clientes y reclamos y quiere utilizarlos para desarrollar un modelo de aprendizaje automático que pueda predecir si un cliente presentará un reclamo de seguro en el próximo año.

## Instalar las dependencias

In [38]:
#!pip install -r requirements.txt

In [39]:
#!pip install pycaret

# Entendimiento inicial de los datos

## Cargar Librerias

In [ ]:
# %matplotlib inline

import re
import random
from collections import Counter

import numpy as np
import pandas as pd

from scipy.stats import kstest

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style='darkgrid')

import scikit-learn
import warnings
warnings.filterwarnings("ignore")

#from pycaret.classification import *
#import shap

## Definición de funciones

## Lectura y validación de datos

In [19]:
# Ruta donde se encuentra alojado el dataset del caso
url_data='../data/'

In [16]:
# Lectura y carga de la data
df=pd.read_csv(url_data+'datos_de_prueba.csv', na_values='?', low_memory=False)

In [17]:
# Validar la correcta lectura del dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   ID del cliente                3000 non-null   int64 
 1   Sexo                          3000 non-null   object
 2   Edad                          3000 non-null   int64 
 3   Ingresos                      3000 non-null   int64 
 4   Tipo de hogar                 3000 non-null   object
 5   Estado civil                  3000 non-null   object
 6   Número de hijos               3000 non-null   int64 
 7   Puntuación de crédito         3000 non-null   int64 
 8   Tipo de trabajo               3000 non-null   object
 9   Educación                     3000 non-null   object
 10  Fecha de inicio de la póliza  3000 non-null   object
 11  Tipo de seguro                3000 non-null   object
 12  Monto del seguro              3000 non-null   int64 
 13  Fecha del reclamo 

Se observa que se tienen 3000 filas y 15 columnas de los tipos int64(7), object(8). Se tiene que modificar el tipo de dato de las variables: ID de cliente, Fecha de inicio de la póliza, Fecha del reclamo para su posterior tratamiento.

No se tienen valores vacios.

In [18]:
# Mostrar los 5 primeros registros
df.head(5)

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo
0,1,Femenino,28,558900,Departamento,Viudo,3,556,Empresario,Preparatoria,2022-03-28,Vida,1806,2022-11-14,558
1,2,Femenino,18,212925,Departamento,Viudo,1,330,Empleado,Posgrado,2022-09-15,Vida,1371,2022-04-25,922
2,3,Femenino,37,838328,Departamento,Casado,3,776,Empresario,Universidad,2022-03-18,Hogar,6751,2022-07-31,3128
3,4,Masculino,25,459616,Casa,Soltero,4,821,Independiente,Universidad,2022-11-14,Hogar,5448,2022-07-29,1493
4,5,Masculino,41,678250,Casa,Casado,4,580,Empresario,Universidad,2023-03-02,Vida,5625,2022-07-08,1907


## Estudiar las dimensiones del dataset

In [20]:
df.shape

(3000, 15)

Se tienen 3000 filas y 15 columnas

In [21]:
df.size

45000

In [24]:
df['ID del cliente'].value_counts()

1       1
2004    1
1995    1
1996    1
1997    1
       ..
1003    1
1004    1
1005    1
1006    1
3000    1
Name: ID del cliente, Length: 3000, dtype: int64

El ID correspondiente al análisis es el ID del cliente, se aprecia que cada ID es único por lo que **cada fila corresponde a un único cliente o reclamo histórico.**

## Evaluar el tipo de problema

Vemos que es un caso de aprendizaje supervisado de clasificación para predecir si un cliente presentará un reclamo de seguro en el próximo año.

In [28]:
df.head(3)

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo
0,1,Femenino,28,558900,Departamento,Viudo,3,556,Empresario,Preparatoria,2022-03-28,Vida,1806,2022-11-14,558
1,2,Femenino,18,212925,Departamento,Viudo,1,330,Empleado,Posgrado,2022-09-15,Vida,1371,2022-04-25,922
2,3,Femenino,37,838328,Departamento,Casado,3,776,Empresario,Universidad,2022-03-18,Hogar,6751,2022-07-31,3128


## Variable objetivo

**Armado del target**

La variable objetivo se construye a partir:
- **0**: Si el año de la fecha de reclamo corresponde al año de la  fecha de inicio de la poliza
- **1**: Si el año de la fecha de reclamo es uno más a la fecha de inicio de la poliza

In [41]:
# Conversión de la fecha para su tratamiento
df['Fecha de inicio de la póliza']=pd.to_datetime(df['Fecha de inicio de la póliza'])
df['Fecha del reclamo']=pd.to_datetime(df['Fecha del reclamo'])

Se observan algunos casos en los que la **fecha de reclamo** es antes a la **fecha de inicio de la poliza** en este sentido estos casos deben ser atentidos por el negocio, por el momento no serán tomados como parte del análisis. Estos serán mapeados con el valor de **-1**

In [47]:
df.loc[df['ID del cliente']==5,:]

,ID del cliente,Sexo,Edad,Ingresos,Tipo de hogar,Estado civil,Número de hijos,Puntuación de crédito,Tipo de trabajo,Educación,Fecha de inicio de la póliza,Tipo de seguro,Monto del seguro,Fecha del reclamo,Monto del reclamo,target
4,5,Masculino,41,678250,Casa,Casado,4,580,Empresario,Universidad,2023-03-02,Vida,5625,2022-07-08,1907,-1


In [43]:
# Se agrega la variable target
df['target'] = df['Fecha del reclamo'].dt.year - df['Fecha de inicio de la póliza'].dt.year
df['target'] = np.where(df['target'] <0, -1, np.where(df['target'] == 1, 1, 0))
df['target'] = df['target'].astype('category')

In [48]:
# Validación
df.target.value_counts()

0     2125
1      441
-1     434
Name: target, dtype: int64

In [50]:
df.target.value_counts(normalize = True)*100

0     70.833333
1     14.700000
-1    14.466667
Name: target, dtype: float64

Se observa que se tiene un 14.7% de valores correspondientes al target==1.

In [51]:
#Removar los valores -1
df = df[df['target'] != -1]